# Use DynamoDB to track the status of Tasks

It many business critical use case, it is necessary to track every task to know which is succeeded, which is failed and which is still in progress. Some advanced users also wants to:

- Each task should be handled by only one worker, you want a concurrency lock mechanism to avoid double consumption.
- For those succeeded tasks, store additional information such as the output of the task and log the success time.
- For those failed task, log the error message for debug, so you can fix the bug and rerun the task.
- For those failed task, you want to get all of failed tasks by one simple query and rerun with the updated business logic.
- For those tasks failed too many times, you don't want to retry them anymore and wants to ignore them.
- Run custom query based on task status for analytics purpose.

With DynamoDB, you can enable this advanced status tracking feature for your application with just a few lines of code. And you can use the "elegant" context manager to wrap around your business logic code and enjoy all the features above.

## Declare Your DynamoDB Status Tracking Table

In [59]:
import pynamodb_mate as pm
from rich import print as rprint

In [60]:
# inherit from the base status enum class and give your status
# a human-readable name and a machine-readable integer
# usually the closer to success, the bigger the integer is
class StatusEnum(pm.patterns.status_tracker.BaseStatusEnum):
    s00_todo = 0
    s03_in_progress = 3
    s06_failed = 6
    s09_success = 9
    s10_ignore = 10


class Tracker(pm.patterns.status_tracker.BaseStatusTracker):
    class Meta:
        # define the table name
        table_name = "pynamodb-mate-example-status-tracker"
        # define the AWS region
        region = "us-east-1"
        # define the billing mode, pay-as-you-go or provisioned
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    # define the index to enable query by status
    # the index name doesn't matter
    status_and_task_id_index = pm.patterns.status_tracker.StatusAndTaskIdIndex()

    # one DynamoDB table can serve multiple jobs
    # if you defined a default job id for the table
    # you don't need to explicitly specify the job id in many API
    # in this specific example, we only have one job called "test-job"
    JOB_ID: str = "test-job"
    # how many digits the max status code have, this ensures that the
    # status can be used in comparison
    STATUS_ZERO_PAD = 3
    # how many retry is allowed before we ignore it
    MAX_RETRY = 3
    # how long the lock will expire
    LOCK_EXPIRE_SECONDS = 900
    # the default status code, means "to do", usually start from 0
    DEFAULT_STATUS = StatusEnum.s00_todo.value
    # the status enum class for this tracker
    STATUS_ENUM = StatusEnum

    def start_job(
        self,
        debug=True,
    ) -> "Tracker":
        """
        This is just an example of how to use :meth:`BaseStatusTracker.start`.

        A job should always have four related status codes:

        - in process status
        - failed status
        - success status
        - ignore status

        If you have multiple type of jobs, I recommend creating multiple
        wrapper functions like this for each type of jobs. And ensure that
        the "ignore" status value is the largest status value among all,
        and use the same "ignore" status value for all type of jobs.
        """
        return self.start(
            in_process_status=StatusEnum.s03_in_progress.value,
            failed_status=StatusEnum.s06_failed.value,
            success_status=StatusEnum.s09_success.value,
            ignore_status=StatusEnum.s10_ignore.value,
            debug=debug,
        )

In [61]:
# Create the table if it doesn't exist
Tracker.create_table(wait=True)

## Initialize a Task

The ``.new(task_id, data)`` method can be used to initialize an task and save to DynamoDB using the ``DEFAULT_STATUS``.

In [62]:
task_id = "t-1"

# create a new task
tracker = Tracker.new(task_id, data={"version": 1})
rprint(tracker.to_dict())

{
    'data': {'version': 1},
    'errors': {},
    'lock_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
    'retry': 0,
    'update_time': datetime.datetime(2023, 1, 2, 16, 23, 9, 528155, tzinfo=datetime.timezone.utc),
    'key': 'test-job____t-1',
    'value': 'test-job____000'
}

The ``.start(in_process_status, failed_status, success_status, ignore_status)`` method is a context manager that automatically update status at begin and the end, and lock the task to avoid concurrent access. We declared a ``.start_job()`` method to wrap the original ``.start(...)`` method to avoid entering too many arguments.

In [63]:
print(f"before the job started, the lock status is {tracker.is_locked()}")

# start the job, it will succeed
with tracker.start_job(debug=True):
    print(f"at begin, the status became {tracker.status_name!r}")
    print("and you can see that the task is locked")
    rprint(tracker.to_dict())

    # do some work
    tracker.set_data({"version": 2})

print(f"at the end, the status became {tracker.status_name!r}")
print("and the lock is released")
rprint(tracker.to_dict())

before the job started, the lock status is False
------ ▶️ start task(job_id='test-job', task_id='t-1', status='s00_todo') ------
🔓 set status 's03_in_progress' and lock the task.
at begin, the status became 's03_in_progress'
and you can see that the task is locked


{
    'data': {'version': 1},
    'errors': {},
    'lock_time': datetime.datetime(2023, 1, 2, 16, 23, 10, 502927, tzinfo=datetime.timezone.utc),
    'retry': 0,
    'update_time': datetime.datetime(2023, 1, 2, 16, 23, 10, 502850, tzinfo=datetime.timezone.utc),
    'key': 'test-job____t-1',
    'lock': '8e7a5b632f3c4930bb497bc9cf36e5e8',
    'value': 'test-job____003'
}

✅ 🔐 task succeeded, set status 's03_in_progress' and unlock the task.
----- ⏹️ end task(job_id='test-job', task_id='t-1', status='s09_success') ------
at the end, the status became 's09_success'
and the lock is released


{
    'data': {'version': 2},
    'errors': {},
    'lock_time': datetime.datetime(2023, 1, 2, 16, 23, 10, 502927, tzinfo=datetime.timezone.utc),
    'retry': 0,
    'update_time': datetime.datetime(2023, 1, 2, 16, 23, 10, 525014, tzinfo=datetime.timezone.utc),
    'key': 'test-job____t-1',
    'value': 'test-job____009'
}

## Error Handling

Let's reset the task and do it one more time, this time the job logic will fail.

- before the task started, the status is still ``s00_todo``
- at begin of the task, the status became ``s03_in_progress``
- at the end of the task, the status become ``s06_failed``
- the task data remains unchanged and the error is logged.

In [64]:
tracker = Tracker.new(task_id, data={"version": 1})

# start the job, it will succeed
with tracker.start_job(debug=True):
    print(f"at begin, the status became {tracker.status_name!r}")
    rprint(tracker.to_dict())

    # do some work
    raise ValueError("something went wrong")
    tracker.set_data({"version": 2})

------ ▶️ start task(job_id='test-job', task_id='t-1', status='s00_todo') ------
🔓 set status 's03_in_progress' and lock the task.
at begin, the status became 's03_in_progress'


{
    'data': {'version': 1},
    'errors': {},
    'lock_time': datetime.datetime(2023, 1, 2, 16, 23, 34, 858572, tzinfo=datetime.timezone.utc),
    'retry': 0,
    'update_time': datetime.datetime(2023, 1, 2, 16, 23, 34, 858429, tzinfo=datetime.timezone.utc),
    'key': 'test-job____t-1',
    'lock': 'bc6fdd293b474baebe21360d04c077c6',
    'value': 'test-job____003'
}

❌ 🔐 task failed, set stats 's06_failed' and unlock the task.
------ ⏹️ end task(job_id='test-job', task_id='t-1', status='s06_failed') ------


ValueError: something went wrong

In [65]:
print(f"at the end, the status became {tracker.status_name!r}")
print("and the error is logged")
rprint(tracker.to_dict())

at the end, the status became 's06_failed'
and the error is logged


{
    'data': {'version': 1},
    'errors': {
        'error': "ValueError('something went wrong')",
        'traceback': 'Traceback (most recent call last):\n  File 
"/Users/sanhehu/Documents/GitHub/pynamodb_mate-project/pynamodb_mate/patterns/status_tracker/impl.py", line 638, in
start\n    try:\n  File "/var/folders/bb/vd34dgxj361gcbkgvmk8_8cw0000gs/T/ipykernel_42365/3315338095.py", line 9, 
in <module>\n    raise ValueError("something went wrong")\nValueError: something went wrong\n'
    },
    'lock_time': datetime.datetime(2023, 1, 2, 16, 23, 34, 858572, tzinfo=datetime.timezone.utc),
    'retry': 1,
    'update_time': datetime.datetime(2023, 1, 2, 16, 23, 34, 905888, tzinfo=datetime.timezone.utc),
    'key': 'test-job____t-1',
    'value': 'test-job____006'
}

## Ignore Task if Failed Too Many Times

You don't want a task that logically can never succeed to fail into a endless loop. In this example, we defined the max retry times is 3. If it failed 3 times in a row, it will be ignored. And if you want to start a task that is ignored, you will see an ``TaskIgnoredError``

In [66]:
# reset the task
tracker = Tracker.new(task_id)

print("at the 0th attempt, the task is:")
rprint(tracker.to_dict())

at the 0th attempt, the task is:


{
    'data': {},
    'errors': {},
    'lock_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
    'retry': 0,
    'update_time': datetime.datetime(2023, 1, 2, 16, 24, 9, 436592, tzinfo=datetime.timezone.utc),
    'key': 'test-job____t-1',
    'value': 'test-job____000'
}

In [67]:
with tracker.start_job():
    raise Exception

------ ▶️ start task(job_id='test-job', task_id='t-1', status='s00_todo') ------
🔓 set status 's03_in_progress' and lock the task.
❌ 🔐 task failed, set stats 's06_failed' and unlock the task.
------ ⏹️ end task(job_id='test-job', task_id='t-1', status='s06_failed') ------


Exception: 

In [68]:
print("at the 1th attempt, the task is:")
print(f"status = {tracker.status_name}")
rprint(tracker.to_dict())

at the 1th attempt, the task is:
status = s06_failed


{
    'data': {},
    'errors': {
        'error': 'Exception()',
        'traceback': 'Traceback (most recent call last):\n  File 
"/Users/sanhehu/Documents/GitHub/pynamodb_mate-project/pynamodb_mate/patterns/status_tracker/impl.py", line 638, in
start\n    try:\n  File "/var/folders/bb/vd34dgxj361gcbkgvmk8_8cw0000gs/T/ipykernel_42365/3027452948.py", line 2, 
in <module>\n    raise Exception\nException\n'
    },
    'lock_time': datetime.datetime(2023, 1, 2, 16, 24, 10, 149582, tzinfo=datetime.timezone.utc),
    'retry': 1,
    'update_time': datetime.datetime(2023, 1, 2, 16, 24, 10, 171897, tzinfo=datetime.timezone.utc),
    'key': 'test-job____t-1',
    'value': 'test-job____006'
}

In [69]:
with tracker.start_job():
    raise Exception

----- ▶️ start task(job_id='test-job', task_id='t-1', status='s06_failed') -----
🔓 set status 's03_in_progress' and lock the task.
❌ 🔐 task failed, set stats 's06_failed' and unlock the task.
------ ⏹️ end task(job_id='test-job', task_id='t-1', status='s06_failed') ------


Exception: 

In [70]:
print("at the 2th attempt, the task is:")
print(f"status = {tracker.status_name}")
rprint(tracker.to_dict())

at the 2th attempt, the task is:
status = s06_failed


{
    'data': {},
    'errors': {
        'error': 'Exception()',
        'traceback': 'Traceback (most recent call last):\n  File 
"/Users/sanhehu/Documents/GitHub/pynamodb_mate-project/pynamodb_mate/patterns/status_tracker/impl.py", line 638, in
start\n    try:\n  File "/var/folders/bb/vd34dgxj361gcbkgvmk8_8cw0000gs/T/ipykernel_42365/3027452948.py", line 2, 
in <module>\n    raise Exception\nException\n'
    },
    'lock_time': datetime.datetime(2023, 1, 2, 16, 24, 10, 988947, tzinfo=datetime.timezone.utc),
    'retry': 2,
    'update_time': datetime.datetime(2023, 1, 2, 16, 24, 11, 8479, tzinfo=datetime.timezone.utc),
    'key': 'test-job____t-1',
    'value': 'test-job____006'
}

In [71]:
with tracker.start_job():
    raise Exception

----- ▶️ start task(job_id='test-job', task_id='t-1', status='s06_failed') -----
🔓 set status 's03_in_progress' and lock the task.
❌ 🔐 task failed 3 times already, set status 's10_ignore' and unlock the task.
------ ⏹️ end task(job_id='test-job', task_id='t-1', status='s10_ignore') ------


Exception: 

In [72]:
print("at the 3th attempt, the task is:")
print(f"status = {tracker.status_name}")
rprint(tracker.to_dict())

at the 3th attempt, the task is:
status = s10_ignore


{
    'data': {},
    'errors': {
        'error': 'Exception()',
        'traceback': 'Traceback (most recent call last):\n  File 
"/Users/sanhehu/Documents/GitHub/pynamodb_mate-project/pynamodb_mate/patterns/status_tracker/impl.py", line 638, in
start\n    try:\n  File "/var/folders/bb/vd34dgxj361gcbkgvmk8_8cw0000gs/T/ipykernel_42365/3027452948.py", line 2, 
in <module>\n    raise Exception\nException\n'
    },
    'lock_time': datetime.datetime(2023, 1, 2, 16, 24, 12, 3836, tzinfo=datetime.timezone.utc),
    'retry': 3,
    'update_time': datetime.datetime(2023, 1, 2, 16, 24, 12, 22500, tzinfo=datetime.timezone.utc),
    'key': 'test-job____t-1',
    'value': 'test-job____010'
}

In [73]:
print("You will see a TaskIgnoredError if you try to start the task again")
with tracker.start_job():
    pass

You will see a TaskIgnoredError if you try to start the task again
----- ▶️ start task(job_id='test-job', task_id='t-1', status='s10_ignore') -----
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/sanhehu/venvs/python/3.8.11/pynamodb_mate_venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/bb/vd34dgxj361gcbkgvmk8_8cw0000gs/T/ipykernel_42365/4135678461.py", line 2, in <module>
    with tracker.start_job():
  File "/Users/sanhehu/.pyenv/versions/3.8.11/lib/python3.8/contextlib.py", line 113, in __enter__
    return next(self.gen)
  File "/Users/sanhehu/Documents/GitHub/pynamodb_mate-project/pynamodb_mate/patterns/status_tracker/impl.py", line 624, in start
pynamodb_mate.patterns.status_tracker.impl.TaskIgnoredError: Task test-job____t-1 retry count already exceeded 3, ignore it.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sanhehu/venvs/python/3.8.11/pynamodb_mate_venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2052, in s